In [61]:
import numpy as np
import random
import nltk
import tensorflow
nltk.download('wordnet')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import json
import pickle
import warnings
warnings.filterwarnings("ignore")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [62]:
intents= json.loads(open('intents.json').read())
print("Processing intents")

Processing intents


In [63]:
nltk.download('punkt')
words = []
classes = []
documents = []
ignore_words = ['?','!',',','.']
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w= nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
            classes.append(intent['tag'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words] 
words= sorted(list(set(words)))
classes = sorted(list(set(classes)))


In [65]:
print (len(documents), "documents")
print (len(classes), "classes")
print (len(words), "Unique stemmed words")

195 documents
8 classes
178 Unique stemmed words


In [66]:
training = []
output = []

output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)


# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


In [67]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [68]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

    # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    # fitting and saving the model
hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5',hist)

print("model created")


Epoch 1/200
39/39 [==============================] - 0s 835us/step - loss: 1.9852 - accuracy: 0.2132
Epoch 2/200
39/39 [==============================] - 0s 1ms/step - loss: 1.7175 - accuracy: 0.3870
Epoch 3/200
39/39 [==============================] - 0s 1ms/step - loss: 1.2295 - accuracy: 0.6479
Epoch 4/200
39/39 [==============================] - 0s 767us/step - loss: 0.9903 - accuracy: 0.6639
Epoch 5/200
39/39 [==============================] - 0s 746us/step - loss: 0.6552 - accuracy: 0.7634
Epoch 6/200
39/39 [==============================] - 0s 827us/step - loss: 0.5136 - accuracy: 0.8092
Epoch 7/200
39/39 [==============================] - 0s 897us/step - loss: 0.4577 - accuracy: 0.8710
Epoch 8/200
39/39 [==============================] - 0s 804us/step - loss: 0.3123 - accuracy: 0.9273
Epoch 9/200
39/39 [==============================] - 0s 812us/step - loss: 0.4268 - accuracy: 0.8643
Epoch 10/200
39/39 [==============================] - 0s 779us/step - loss: 0.2473 - accuracy: 

In [69]:
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [70]:
from tensorflow.keras.models import load_model
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

with open('intents.json') as json_data:
    intents = json.load(json_data)

model=load_model('chatbot_model.h5')

Loading Pickle.....


In [71]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    results = model.predict(np.array([bow(sentence, words)]))[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) 
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return random.choice(i['responses'])

            results.pop(0)
            

ERROR_THRESHOLD = 0.25


In [72]:
flag= True
while (flag==True) :
  input_data=input("You: ")
  input_data= input_data.lower()
  if input_data not in ['bye','goodbye','quit']:
    answer = response(input_data)
    print("Bot:", answer)
  else:
    flag=False
    print("Bot: Bye! take care")



You: Hey
Bot: hello
You: Not able to view videos
Bot: please check your internet connection and try again
You: Trouble saving videos
Bot: please make some memory free in your mobile and try again
You: Thankyou
Bot: You're welcome
You: bye
Bot: Bye! take care
